In [1]:
import pandas as pd
import hops.hdfs as hdfs
import tarfile
import tables
import os
import string
fs_handle = hdfs.get_fs()

In [2]:
print(os.environ['PDIR'])

/srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349


In [3]:
def get_segments_timbre(h5,songidx=0):
    
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_timbre[h5.root.analysis.songs.cols.idx_segments_timbre[songidx]:,:]
    return h5.root.analysis.segments_timbre[h5.root.analysis.songs.cols.idx_segments_timbre[songidx]:
                                            h5.root.analysis.songs.cols.idx_segments_timbre[songidx+1],:]

def get_year(h5,songidx=0):

    return h5.root.musicbrainz.songs.cols.year[songidx]

def get_artist_id(h5,songidx=0):

    return h5.root.metadata.songs.cols.artist_id[songidx]

In [4]:
def read_target_track_id_list():
    # Read the track ids with top 100 tags
    with fs_handle.open_file(hdfs.project_path() + 'Resources/track_id_with_top_100_tag.txt', 'r') as fd:
        data = fd.read()
    
    return data.split('\n')[:-1]

def put_target_track_into_buckets(target_track_id_list):
    
    buckets = {}
    
    for upper_letter in string.ascii_uppercase:
         buckets[upper_letter] = filter(lambda track_id: track_id[2] == upper_letter, target_track_id_list)
            
    return buckets

In [6]:
target_track_id_list = read_target_track_id_list()
print(len(target_track_id_list))

target_tracks_buckets = put_target_track_into_buckets(target_track_id_list)
print(len(target_tracks_buckets))

369047
26


In [35]:
help(fs_handle.copy)
len(target_tracks_buckets['C'])

Help on method copy in module pydoop.hdfs.fs:

copy(self, from_path, to_hdfs, to_path) method of pydoop.hdfs.fs.hdfs instance
    Copy file from one filesystem to another.
    
    :type from_path: str
    :param from_path: the path of the source file
    :type to_hdfs: :class:`hdfs`
    :param to_hdfs: destination filesystem
    :type to_path: str
    :param to_path: the path of the destination file
    :raises: :exc:`~exceptions.IOError`



14337

In [42]:
def move_h5_file_to_local_path(tar_file_handle, track_id, feature_file_handle):
    
    extract_file_name = track_id[2] + '/' + \
                        track_id[3] + '/' + \
                        track_id[4] + '/' + \
                        track_id + '.h5'
    
#     print('Extracting', extract_file_name)
    f_obj = tar_file_handle.extractfile(extract_file_name)
    
#     print('Writing %s into hdfs path' % (track_id))
    extract_h5_file_path = hdfs.project_path() + "Resources/extract_h5_tmp.h5"
    with fs_handle.open_file(extract_h5_file_path, flags='w') as fd:
        fd.write(f_obj.read())
    
    f_obj.close()
    
#     print('Moving %s into local path' % (track_id))
#     copy the h5 file from hdfs filesystem to local filesystem
    h5_lcoal_path = os.environ['PDIR']
    hdfs.hdfs.get(extract_h5_file_path, h5_lcoal_path)
    
#     print('Reading %s from local path' % (track_id))
#     Extract some info from the h5 file
    with tables.open_file(os.environ['PDIR'] + '/extract_h5_tmp.h5', mode="r") as h5_file_handle:
#         print('segments_timbre shape:', get_segments_timbre(h5_file_handle).shape)
        
        mfcc_vec = get_segments_timbre(h5_file_handle).reshape(-1).tolist()
        mfcc_vec.insert(0, track_id)
        
        to_str_mfcc_vec = list(map(lambda v: str(v), mfcc_vec))
        feature_file_handle.write(','.join(to_str_mfcc_vec) + '\n')
#         print()
    
#     print('Removing extract_h5_tmp.h')
#     remove the tmp h5 file to ensure upcoming file move
    os.system('rm %s/%s' % (os.environ['PDIR'], 'extract_h5_tmp.h5'))
    
#     print('Leaving move_h5_file_to_local_path')
    
def extract_info_from_tgz(bucket):
    
    file_path = 'hdfs:///Projects/labs/million_song/huge_dataset/%s.tar.gz' % (bucket)
    
    print('Moving %s.tar.gz into local path' % (bucket))
    hdfs.hdfs.get(file_path, os.environ['PDIR'])
    print('Finished moving %s.tar.gz into local path' % (bucket))
#     fd = fs_handle.open_file(file_path, 'r')

    print('Open %s.tar.gz file' % (bucket))
    tar_file_handle = tarfile.open(os.environ['PDIR'] + '/%s.tar.gz' % (bucket), mode="r")
    
#     print('extracting h5 files')
#     h5_file_names = filter(lambda f: f.endswith('.h5'), tar_file_handle.getnames())
    
    feature_file_handle = fs_handle.open_file(hdfs.project_path() + "Resources/audio_feactures/%s/mfcc_vectors.txt" % (bucket), 
                             flags='w')

    for h5_file_name in target_tracks_buckets[bucket]:
        move_h5_file_to_local_path(tar_file_handle, h5_file_name, feature_file_handle)
    
    feature_file_handle.close()
    tar_file_handle.close()
    print('removing %s' % (os.environ['PDIR'] + '/%s.tar.gz' % (bucket)))
    os.system('rm %s' % (os.environ['PDIR'] + '/%s.tar.gz' % (bucket)))
    
    print('Leaving extract_info_from_tgz')

In [ ]:
extract_info_from_tgz('A')

Moving A.tar.gz into local path
Finished moving A.tar.gz into local path
Open A.tar.gz file


In [ ]:
print("Finished")

In [ ]:
total_h5_file_cnt = 0

for bucket, tracks in target_tracks_buckets.items():
    print('Working on %s' % bucket)
    h5_file_cnt = extract_info_from_tgz(bucket)
    print('h5_file_cnt', h5_file_cnt)
    
    total_h5_file_cnt += h5_file_cnt
    
print('total_h5_file_cnt', total_h5_file_cnt)

Working on A
extracting h5 files
removing /srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349/A.tar.gz
('h5_file_cnt', 39100)
Working on C
extracting h5 files
removing /srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349/C.tar.gz
('h5_file_cnt', 38611)
Working on B
extracting h5 files
removing /srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349/B.tar.gz
('h5_file_cnt', 38265)
Working on E
extracting h5 files
removing /srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349/E.tar.gz
('h5_file_cnt', 38466)
Working on D
extracting h5 files
removing /srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349/D.tar.gz
('h5_file_cnt', 38825)
Working on G
extracting h5 files
removing /srv/hops/staging/private_dirs/a584dcd0f2d97115248b094f8bb4cca326af95d74ddbbef145147b1e0ec89349/G.tar.gz
('h

In [30]:
logfile = hdfs.project_path() + "Resources/A/file.txt"
fd = fs_handle.open_file(logfile, flags='w')

In [31]:
fd.write('Hello HopsWorksjhjashdas')

24

In [32]:
fd.write('Hello HopsWorksasdas')

20

In [33]:
fd.close()

True